In [73]:
import pandasdmx as sdmx
import pandas as pd
import sys

### Choose and connect to an SDMX web service

In [39]:
ecb = sdmx.Request('ECB')

### Configure the HTTP connection

In [40]:
ecb_via_proxy = sdmx.Request(
    'ECB',
    proxies={'http': 'http://1.2.3.4:5678'}
)

In [41]:
ecb_via_proxy.session.proxies

{'http': 'http://1.2.3.4:5678'}

### Cache HTTP responses and parsed objects

In [42]:
ecb_with_cache = sdmx.Request(
    'ECB',
    backend='sqlite',
    fast_save=True,
    expire_after=600,
)

### Get information about the source’s data flows

In [43]:
flow_msg = ecb.dataflow()

In [44]:
flow_msg.response.url

'https://data-api.ecb.europa.eu/service/dataflow/ECB/latest'

In [45]:
flow_msg.response.headers

{'Server': 'myracloud', 'Date': 'Wed, 27 Mar 2024 09:04:24 GMT', 'Content-Type': 'application/vnd.sdmx.structure+xml;version=2.1', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Expires': 'Wed, 27 Mar 2024 09:04:54 GMT', 'cache-control': 'max-age=30', 'ETag': '"myra-6f8efcd9"'}

In [46]:
flow_msg

<pandasdmx.StructureMessage>
  <Header>
    id: 'IREF302684'
    prepared: '2024-03-27T09:04:24+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (95): AME BKN BLS BNT BOP BPS BSI BSP CBD CBD2 CCP...
  DataStructureDefinition (73): ECB_AME1 ECB_BKN1 ECB_BLS1 ECB_BOP_BNT ...

In [47]:
flow_msg.dataflow.BOP

<DataflowDefinition ECB:BOP(1.0): Euro Area Balance of Payments and International Investment Position Statistics>

### Convert metadata to pandas.Series

In [48]:
dataflows = sdmx.to_pandas(flow_msg.dataflow)

print(len(dataflows))

dataflows.head()

95


AME                                                AMECO
BKN                                 Banknotes statistics
BLS                       Bank Lending Survey Statistics
BNT        Shipments of Euro Banknotes Statistics (ESCB)
BOP    Euro Area Balance of Payments and Internationa...
dtype: object

In [49]:
dataflows[dataflows.str.contains('exchange', case=False)]

EXR                              Exchange Rates
FXI                 Foreign Exchange Statistics
SEE    Securities exchange - Trading Statistics
dtype: object

### Extract the metadata related to a data flow

In [50]:
exr_msg = ecb.dataflow('EXR')

In [51]:
exr_msg.response.url

'https://data-api.ecb.europa.eu/service/dataflow/ECB/EXR/latest?references=all'

In [52]:
exr_msg

<pandasdmx.StructureMessage>
  <Header>
    id: 'IREF302685'
    prepared: '2024-03-27T09:04:26+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (2): 21e97b57-5950-eaab-eead-1534306b28af 53A341E8-D48...
  CategoryScheme (2): WDC_NAVI WDC_NAVI_OLD
  Codelist (11): CL_COLLECTION CL_CURRENCY CL_DECIMALS CL_EXR_SUFFIX CL...
  ConceptScheme (1): ECB_CONCEPTS
  ContentConstraint (1): EXR_CONSTRAINTS
  DataflowDefinition (1): EXR
  DataStructureDefinition (1): ECB_EXR1
  AgencyScheme (1): AGENCIES

In [53]:
exr_flow = exr_msg.dataflow.EXR

In [54]:
# Show the data structure definition referred to by the data flow
dsd = exr_flow.structure
dsd

<DataStructureDefinition ECB:ECB_EXR1(1.0): Exchange Rates>

In [55]:
# The same object instance is accessible from the StructureMessage
dsd is exr_msg.structure.ECB_EXR1

True

In [56]:
# Explore the DSD
dsd.dimensions.components

[<Dimension FREQ>,
 <Dimension CURRENCY>,
 <Dimension CURRENCY_DENOM>,
 <Dimension EXR_TYPE>,
 <Dimension EXR_SUFFIX>,
 <TimeDimension TIME_PERIOD>]

In [57]:
dsd.attributes.components

[<DataAttribute TIME_FORMAT>,
 <DataAttribute OBS_STATUS>,
 <DataAttribute OBS_CONF>,
 <DataAttribute OBS_PRE_BREAK>,
 <DataAttribute OBS_COM>,
 <DataAttribute BREAKS>,
 <DataAttribute COLLECTION>,
 <DataAttribute COMPILING_ORG>,
 <DataAttribute DISS_ORG>,
 <DataAttribute DOM_SER_IDS>,
 <DataAttribute PUBL_ECB>,
 <DataAttribute PUBL_MU>,
 <DataAttribute PUBL_PUBLIC>,
 <DataAttribute UNIT_INDEX_BASE>,
 <DataAttribute COMPILATION>,
 <DataAttribute COVERAGE>,
 <DataAttribute DECIMALS>,
 <DataAttribute NAT_TITLE>,
 <DataAttribute SOURCE_AGENCY>,
 <DataAttribute SOURCE_PUB>,
 <DataAttribute TITLE>,
 <DataAttribute TITLE_COMPL>,
 <DataAttribute UNIT>,
 <DataAttribute UNIT_MULT>]

In [58]:
dsd.measures.components

[<PrimaryMeasure OBS_VALUE>]

In [59]:
# Show a codelist referenced by a dimension, containing a superset
# of existing values
cl = dsd.dimensions.get('FREQ').local_representation.enumerated
cl

<Codelist ECB:CL_FREQ(1.0) (10 items): Frequency code list>

In [60]:
# Again, the same object can be accessed directly
cl is exr_msg.codelist.CL_FREQ

True

In [61]:
sdmx.to_pandas(cl)

,name,parent
CL_FREQ,,
A,Annual,CL_FREQ
B,Daily - businessweek,CL_FREQ
D,Daily,CL_FREQ
E,Event (not supported),CL_FREQ
H,Half-yearly,CL_FREQ
M,Monthly,CL_FREQ
N,Minutely,CL_FREQ
Q,Quarterly,CL_FREQ
S,"Half-yearly, semester (value introduced in 200...",CL_FREQ


### Internationalisation

In [62]:
ecb.default_locale

'en'

In [63]:
ecb.default_locale = "fr"

In [64]:
# Note that this setting is global, not per Request instance.
insee_flows = sdmx.Request('insee').dataflow()
sdmx.to_pandas(insee_flows.dataflow).head()

BALANCE-PAIEMENTS                                    Balance des paiements
CHOMAGE-TRIM-NATIONAL    Chômage, taux de chômage par sexe et âge (sens...
CLIMAT-AFFAIRES            Indicateurs synthétiques du climat des affaires
CNA-2010-CONSO-MEN       Consommation des ménages - Résultats par produ...
CNA-2010-CONSO-SI        Dépenses de consommation finale par secteur in...
dtype: object

In [65]:
ecb.default_locale = "en"
sdmx.to_pandas(insee_flows.dataflow).head()

BALANCE-PAIEMENTS                                      Balance of payments
CHOMAGE-TRIM-NATIONAL    Unemployment, unemployment rate and halo by se...
CLIMAT-AFFAIRES                      Business climate composite indicators
CNA-2010-CONSO-MEN       Households' consumption - Results by product, ...
CNA-2010-CONSO-SI        Final consumption expenditure by institutional...
dtype: object

## Select and query data from a dataflow

### Construct a selection key for a query

In [66]:
key = dict(CURRENCY=['USD', 'JPY'])

params = dict(startPeriod='2016')

### Query data

In [67]:
ecb = sdmx.Request('ECB', backend='memory')

data_msg = ecb.data('EXR', key=key, params=params)

# Generic data was returned
data_msg.response.headers['content-type']

2024-03-27 10:04:27,394 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


'application/vnd.sdmx.genericdata+xml;version=2.1'

In [68]:
# Number of bytes in the cached response
bytes1 = sys.getsizeof(ecb.session.cache.responses.popitem()[1].content)

bytes1

1099575

In [69]:
ss_msg = ecb.data('EXR', key=key, params=params, dsd=dsd)
ss_msg.response.request.headers['accept']

'application/vnd.sdmx.structurespecificdata+xml;version=2.1'

In [70]:
ss_msg.response.headers['content-type']

'application/vnd.sdmx.structurespecificdata+xml;version=2.1'

In [71]:
bytes2 = sys.getsizeof(ecb.session.cache.responses.popitem()[1].content)
bytes2 / bytes1

0.34012959552554395

In [72]:
data = data_msg.data[0]
print(list(data.series.keys())[5])
set(series_key.FREQ for series_key in data.series.keys())

(FREQ=D, CURRENCY=USD, CURRENCY_DENOM=EUR, EXR_TYPE=SP00, EXR_SUFFIX=A)


{<KeyValue: FREQ=A>,
 <KeyValue: FREQ=D>,
 <KeyValue: FREQ=H>,
 <KeyValue: FREQ=M>,
 <KeyValue: FREQ=Q>}

## Convert data to pandas

### Select columns using the model API